In [ ]:
import tweepy
import json
from pymongo import MongoClient

In [ ]:
client = MongoClient(port=27017)
db=client.twitter

In [ ]:
WORDS = ['#bigdata', '#AI', '#datascience', '#machinelearning', '#ml', '#iot']

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [ ]:
class StreamListener(tweepy.StreamListener):    
    #This is a class provided by tweepy to access the Twitter Streaming API. 

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False
     
     
    def on_data(self, data):
        #This is the meat of the script...it connects to your mongoDB and stores the tweet
        try:
             
            
            # Use twitterdb database. If it doesn't exist, it will be created.
             
    
            # Decode the JSON from Twitter
            datajson = json.loads(data)
            text=''
            if 'retweeted_status' in datajson:
                if 'extended_tweet' in datajson['retweeted_status']:
                    text = datajson['retweeted_status']['extended_tweet']['full_text']
                else:
                    text = datajson['retweeted_status']['text']
            elif 'quoted_status' in datajson:
                if 'extended_tweet' in datajson['quoted_status']:
                    text = datajson['quoted_status']['extended_tweet']['full_text']
                else:
                    text = datajson['text']
            if text=='':
                text = datajson['text']
            
            #grab the 'created_at' data from the Tweet to use for display
            created_at = datajson['created_at']

            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(created_at))
            hashtag=[]
            for h in datajson['entities']['hashtags']:
                hashtag.append(h['text'])
            res={}
            res['text']=text
            res['created_at']=created_at
            res['_id'] = datajson['id_str']
            res['postid'] = datajson['id']
            res['link']= 'https://twitter.com/' + datajson['user']['screen_name'] + '/status/'+ datajson['id_str']
            res['postauthor']= datajson['user']['name']
            res['userid']= str(datajson['user']['id'])
            res['hashtag'] = [tag for tag in text.split() if tag.startswith('#')]
            #insert the data into the mongoDB into a collection called twitter_search
            #if twitter_search doesn't exist, it will be created.
            
            #print(datajson)
            db.tweets.insert_one(res)
        except Exception as e:
           print(e)

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS)